In [15]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
aptitude is already the newest version (0.8.13-3ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
mecab is already installed at the requested version (0.996-14build9)
libmecab-dev is already installed at the requested version (0.996-14build9)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-3)
git is already installed at the requested version (1:2.34.1-1ubuntu1.11)
make is already installed at the requested version (4.3-4.1build1)
curl is already installed at the requested version (7.81.0-1ubuntu1.17)
xz-utils is already installed at the requested version (5.2.5-2ubuntu1)
file is already installed at the requested version (1:5.41-3ubuntu0.1)
mecab is already installed at the requested version (0.996-14build9)
libmecab-dev is already installed at the requested version (0.996-14build9)
mecab-ipadic-utf8 is already installed at the re

In [2]:
import MeCab
import tensorflow as tf


In [30]:
#Tokenisation

from sklearn.utils import shuffle

train_strings = []
validation_strings = []
test_strings = []
train_labels = []
validation_labels = []
test_labels = []
mecab = MeCab.Tagger("-Owakati")
for file_name, line_list, label_list in zip(["train.txt", "validation.txt", "test.txt"], [train_strings, validation_strings, test_strings], [train_labels, validation_labels, test_labels]):
    with open(file_name, "r", encoding="utf-8") as f:
        lines = f.readlines()
        lines = [line.strip().split("\t") for line in lines]
        mecab = MeCab.Tagger("-Owakati")
        for i in range(len(lines)):
            line = lines[i][0]
            Writer_joy = lines[i][4] #Writer_joy signifies the level of joy of the sentence
            line = mecab.parse(line).strip()
            line_list.append(line)
            if Writer_joy == "0":
                label_list.append(0) #negative
            else:
                label_list.append(1) #positive

train_strings, train_labels = shuffle(train_strings, train_labels)
validation_strings, validation_labels = shuffle(validation_strings, validation_labels)
test_strings, test_labels = shuffle(test_strings, test_labels)
train_strings, train_labels = train_strings[:int(len(train_strings))], train_labels[:int(len(train_labels))]
validation_strings, validation_labels = validation_strings[:int(len(validation_strings))], validation_labels[:int(len(validation_labels))]
test_strings, test_labels = test_strings[:int(len(test_strings)/1)], test_labels[:int(len(test_labels)/1)]





print(len(train_strings), len(validation_strings), len(test_strings))


30000 2500 2500


In [25]:
for i in range(10):
    print(train_strings[i], train_labels[i])

すごい スピード 感 0
各人 が やり たい 仕事 を 自由 に 意見 でき 、 それ が ある程度 叶う 環境 で あれ ば まだ わかる が 、 まず お前 ら が その 環境 を 作る 努力 を し た の か と 。 夜勤 に ぶち込ん で 酷使 し とい て そんな 向上心 が 勝手 に 芽生える と 思う な よ と 。 0
ちなみに 昨日 の 夜 は 今日 の 仕事 が 嫌 すぎ て アマゾン で ポチッ て しまっ た 。 の やっ さん と 大 魔王 とけん まち ゃんのねんどろいど … 。 振込 に 行か なく ちゃ … ♪ 1
に ゃんにゃん ( 笑 ) 1
仕事 いっぱい もらえ た ので 頑張り ます 1
アカン 、 昨晩 から ずっと み ゅうみゅうと 子猫 の 鳴き声 が 響い て い て めちゃくちゃ 気 に なる … 何 も なけれ ば いい ん だ けど … 0
掃除 は 無料 で スッキリ 感 を 味わえる エンタメ で ある 。 1
謎 理論 0
夢 の 中 で 悲しく て 辛い こと が あっ て ずっと ごめん 許し て っ て 泣い て 叫ん で た 気 が する 。 頭 が 痛い 。 0
もう 勘弁 し て ください (‾▽‾;)" 0


In [55]:
TRAIN = [line.split(" ") for line in train_strings]
VALIDATION = [line.split(" ") for line in validation_strings]
TEST = [line.split(" ") for line in test_strings]

In [57]:
total = 0
for i in range(len(TRAIN)):
    total += len(TRAIN[i])
print(total/len(TRAIN))

19.282233333333334


Bigrams with binary encoding

In [31]:
#text vectorisation

from tensorflow import keras
from keras import layers

text_vectorization = layers.TextVectorization(max_tokens=20000, ngrams=2, output_mode="multi_hot", split="whitespace")
text_vectorization.adapt(train_strings)

In [37]:

def prepare_dataset(texts, labels, batch_size=256):
    dataset = tf.data.Dataset.from_tensor_slices((texts, labels))
    dataset = dataset.map(lambda x, y: (text_vectorization(x), y))
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

train_dataset = prepare_dataset(train_strings, train_labels)
validation_dataset = prepare_dataset(validation_strings, validation_labels)


max_tokens = 20000
inputs = keras.Input(shape=(max_tokens,),)
x = layers.Dense(16, activation="relu")(inputs)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
history = model.fit(
    train_dataset,
    batch_size=256,
    epochs=10,
    validation_data=validation_dataset,
    callbacks = [keras.callbacks.EarlyStopping(monitor="val_loss", patience=2)]
)


Epoch 1/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 47s 383ms/step - accuracy: 0.6191 - loss: 0.6624 - val_accuracy: 0.6008 - val_loss: 0.6979
Epoch 2/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 45s 379ms/step - accuracy: 0.7352 - loss: 0.5442 - val_accuracy: 0.6388 - val_loss: 0.6894
Epoch 3/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 45s 385ms/step - accuracy: 0.7791 - loss: 0.4843 - val_accuracy: 0.6376 - val_loss: 0.7210
Epoch 4/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 45s 382ms/step - accuracy: 0.8064 - loss: 0.4366 - val_accuracy: 0.6380 - val_loss: 0.7486


TF-IDF

In [38]:
#text vectorisation
text_vectorization = layers.TextVectorization(max_tokens=20000, ngrams=2, output_mode="tf_idf", split="whitespace")
text_vectorization.adapt(train_strings)

In [42]:
#training



train_dataset = prepare_dataset(train_strings, train_labels)
validation_dataset = prepare_dataset(validation_strings, validation_labels)


max_tokens = 20000
inputs = keras.Input(shape=(max_tokens,),)
x = layers.Dense(16, activation="relu")(inputs)
x = layers.Dense(16, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
history = model.fit(
    train_dataset,
    batch_size=256,
    epochs=10,
    validation_data=validation_dataset,
    callbacks = [keras.callbacks.EarlyStopping(monitor="val_loss", patience=2)]
)


Epoch 1/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 47s 383ms/step - accuracy: 0.6207 - loss: 0.6575 - val_accuracy: 0.5920 - val_loss: 0.7363
Epoch 2/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 45s 383ms/step - accuracy: 0.7542 - loss: 0.5186 - val_accuracy: 0.6284 - val_loss: 0.7240
Epoch 3/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 45s 378ms/step - accuracy: 0.8129 - loss: 0.4269 - val_accuracy: 0.6356 - val_loss: 0.7901
Epoch 4/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 45s 380ms/step - accuracy: 0.8455 - loss: 0.3602 - val_accuracy: 0.6260 - val_loss: 0.9466
